In [17]:
from nsetools import Nse
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import datetime as dt
from datetime import datetime
import pickle5 as pickle
import threading
import os
import json

In [18]:
url = "https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp"
now = datetime.now().strftime("%H:%M:%S")
history = list()

#Create dataframe
strikes = [50*i for i in range(220,281)]
df=pd.DataFrame(columns = ['strike_price'], index = strikes)
for i in strikes:
    df.loc[i,'strike_price'] = i

In [19]:
def get_soup(url):
    response = requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
    if(response.status_code != 200):
        print('Illegal URL')
        return None
    soup = BeautifulSoup(response.text,'lxml')
    print("Soup generation. 200!")
    return soup

In [20]:
def get_nifty(soup):
    try:
        val1 = soup.find_all('span')[0].text.split()[-1]
        val2 = soup.find_all('span')[1].text.split()[-2]
        return ({'nifty':val1,'time':val2})
    except:
        print("Bad request")
        return None

In [21]:
def insert_values(soup, nifty, dataframe):
    cols = [
        'CE_OI_@'+nifty['time'],
        'CE_CHOI_@'+nifty['time'],
        'CE_IV_@'+nifty['time'],
        'CE_LTP_@'+nifty['time'],
        'PE_OI_@'+nifty['time'],
        'PE_CHOI_@'+nifty['time'],
        'PE_IV_@'+nifty['time'],
        'PE_LTP_@'+nifty['time'],
        'NIFTY@'+nifty['time'],
        'CE_VOL_@'+nifty['time'],
        'PE_VOL_@'+nifty['time']
    ]
    
    for col in cols:
        dataframe[col] = np.nan
    
    for one in soup.find_all('div',{'class':'opttbldata'})[0].find_all('tr')[2:-1]:
        one = one.find_all('td')
        strike = one[11].text
        if(int(float(strike)) not in list(df['strike_price'])):
            continue
        values = dict()
        values[cols[0]] = one[1].text
        values[cols[1]] = one[2].text
        values[cols[2]] = one[4].text
        values[cols[3]] = one[5].text
        values[cols[4]] = one[21].text
        values[cols[5]] = one[20].text
        values[cols[6]] = one[18].text
        values[cols[7]] = one[17].text
        values[cols[8]] = nifty['nifty']
        values[cols[9]] = one[3].text
        values[cols[10]] = one[19].text
    
        for key in values:
            values[key] = values[key].replace(" ","")
            values[key] = values[key].replace("\t","")
            values[key] = values[key].replace("\n","")
            values[key] = values[key].replace("\r","")
            values[key] = values[key].replace("\n","")
            values[key] = values[key].replace(",","")
            try: 
                values[key] = float(values[key])
            except:
                values[key] = np.nan
            
            dataframe.loc[int(float(strike)),key] = values[key]

    return dataframe

In [22]:
def get_sum_n_atm_options(dataframe, size, nifty):
    nifty['nifty'] = int(float(nifty['nifty']))
    strike = nifty['nifty']-nifty['nifty']%50
    curtime = nifty['time']
    
    subdf = df.loc[strike-size*50:strike+size*50,'CE_OI_@'+curtime:]
    sums = dict()
    sums['CE_CHOI'] = subdf.loc[:,'CE_CHOI_@'+curtime].sum()
    sums['PE_CHOI'] = subdf.loc[:,'PE_CHOI_@'+curtime].sum()
    sums['CE_OI'] = subdf.loc[:,'CE_OI_@'+curtime].sum()
    sums['PE_OI'] = subdf.loc[:,'PE_OI_@'+curtime].sum()
    sums['ce_ratio']=sums['CE_CHOI']/sums['CE_OI']
    sums['pe_ratio']=sums['PE_CHOI']/sums['PE_OI']
    sums['pe_ce_choi_diff'] = subdf.loc[:,'PE_CHOI_@'+curtime].sum() - subdf.loc[:,'CE_CHOI_@'+curtime].sum()
    sums['pcr'] = sums['PE_OI']/sums['CE_OI']
    sums['nifty'] = nifty['nifty']
    sums['time'] = nifty['time']
    
    return sums

In [ ]:
soup = get_soup(url)
nifty = get_nifty(soup)
print(nifty)

In [ ]:
df=insert_values(soup, nifty, df)
sumsHistory = get_sum_n_atm_options(df, 5, nifty)
print(sumsHistory)

In [ ]:
if(len(history)>0):
    if history[-1]['CE_CHOI'] == sumsHistory['CE_CHOI'] and history[-1]['PE_CHOI'] == sumsHistory['PE_CHOI']:
            print("Similar data exists \nTime of exec :", nifty['time'])
    else:
        print("Unique data entered!\n")
        history.append(sumsHistory)
else:
    print("Unique data entered!\n")
    history.append(sumsHistory)
history

In [24]:
def automation(df,history):
    soup = get_soup(url)
    nifty = get_nifty(soup)
    print(nifty)
    
    df=insert_values(soup, nifty, df)
    sumsHistory = get_sum_n_atm_options(df, 5, nifty)
    print(sumsHistory)
    
    if(len(history)>0):
        if history[-1]['CE_CHOI'] == sumsHistory['CE_CHOI'] and history[-1]['PE_CHOI'] == sumsHistory['PE_CHOI']:
                print("Similar data exists \nTime of exec :", nifty['time'])
        else:
            print("Unique data entered!\n")
            history.append(sumsHistory)
    else:
        print("Unique data entered!\n")
        history.append(sumsHistory)
    print(history)
    
    with open("dictionary.txt", "wb") as fp:
        pickle.dump(history, fp)

    df.to_csv('2020.csv',index=False)
    
    return df,history
    

In [26]:
df,history = automation(df,history)

Soup generation. 200!
{'nifty': '12968.95', 'time': '15:30:28'}
{'CE_CHOI': 5125050.0, 'PE_CHOI': 3782400.0, 'CE_OI': 12327600.0, 'PE_OI': 10468875.0, 'ce_ratio': 0.4157378565170836, 'pe_ratio': 0.3612995665723394, 'pe_ce_choi_diff': -1342650.0, 'pcr': 0.849222476394432, 'nifty': 12968, 'time': '15:30:28'}
Similar data exists 
Time of exec : 15:30:28
[{'CE_CHOI': 5125050.0, 'PE_CHOI': 3782400.0, 'CE_OI': 12327600.0, 'PE_OI': 10468875.0, 'ce_ratio': 0.4157378565170836, 'pe_ratio': 0.3612995665723394, 'pe_ce_choi_diff': -1342650.0, 'pcr': 0.849222476394432, 'nifty': 12968, 'time': '15:30:28'}]


In [27]:
df

,strike_price,CE_OI_@15:30:28,CE_CHOI_@15:30:28,CE_IV_@15:30:28,CE_LTP_@15:30:28,PE_OI_@15:30:28,PE_CHOI_@15:30:28,PE_IV_@15:30:28,PE_LTP_@15:30:28,NIFTY@15:30:28,CE_VOL_@15:30:28,PE_VOL_@15:30:28
11000,11000,150.0,150.0,NaN,2022.75,240675.0,45750.0,49.49,0.65,12968.95,2.0,3905.0
11050,11050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12968.95,NaN,NaN
11100,11100,NaN,NaN,NaN,NaN,225.0,225.0,NaN,0.75,12968.95,NaN,74.0
11150,11150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12968.95,NaN,NaN
11200,11200,NaN,NaN,NaN,NaN,15600.0,14475.0,NaN,1.00,12968.95,NaN,1175.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13800,13800,1018425.0,587175.0,20.47,1.90,225.0,NaN,NaN,838.05,12968.95,34014.0,2.0
13850,13850,10200.0,10050.0,21.09,1.95,NaN,NaN,NaN,NaN,12968.95,511.0,NaN
13900,13900,365625.0,121425.0,21.99,1.75,75.0,75.0,NaN,920.60,12968.95,13801.0,4.0
13950,13950,3450.0,1800.0,22.75,1.70,150.0,150.0,NaN,1131.80,12968.95,206.0,8.0


In [28]:
history

[{'CE_CHOI': 5125050.0,
  'PE_CHOI': 3782400.0,
  'CE_OI': 12327600.0,
  'PE_OI': 10468875.0,
  'ce_ratio': 0.4157378565170836,
  'pe_ratio': 0.3612995665723394,
  'pe_ce_choi_diff': -1342650.0,
  'pcr': 0.849222476394432,
  'nifty': 12968,
  'time': '15:30:28'}]